Here we fine-tune the encoder-only DilstilBERT model, a model trained using a knowledge distillation process from BERT. The model is domain adapated (fine-tuned) on the IMDB dataset for an ultimate masked language model task that fills [MASK] tokens in sentences with an empsasis on terminology common to movies. We publish this model to the Hugging Face hub with the name [DistilBERT-DeNiro](https://huggingface.co/MarioBarbeque/DistilBERT-DeNiro/tree/main).

Rather than using the simpler 🤗 `Trainer` API, we will build our training loop directly in PyTorch for more control over simple details. In the Azure Databricks notebook where we have completed previous model training ([RoBERTa-base-DReiFT](https://huggingface.co/MarioBarbeque/RoBERTa-base-DReiFT), for example), we found that configuring single-node multi-GPU distributed training with PyTorch's `DistributedDataParallel` and `DistributedSampler` classes was tediously tricky in the Databricks notebook environment. For models like RoBERTa-base-DReiFT, we instead made use of the simpler, self-contained 🤗 `Trainer` API and wrapped it in the PySpark `TorchDistributor` class which is designed to orchestrate distributed training of PyTorch models through Apache Spark.

For the training of this model, we use a single-node single Nvidia T4 GPU compute instance to train our model locally without any distributed processes.

## Preprocessing and Prep for Training

In [0]:
# evaluate GPU memory of this single node with an Nvidia T4 GPU
import torch

def mem_status(): 
    if torch.cuda.is_available():
        gpus = torch.cuda.device_count()
        print("Memory status: ")
        for i in range(gpus):
            properties = torch.cuda.get_device_properties(i)
            total_memory = properties.total_memory / (1024 ** 3)  # Convert to GB
            allocated_memory = torch.cuda.memory_allocated(i) / (1024 ** 3)  # Convert to GB
            reserved_memory = torch.cuda.memory_reserved(i) / (1024 ** 3)  # Convert to GB
            available_memory = total_memory - reserved_memory
            print(f"GPU {i}:")
            print(f"  Total memory: {total_memory:.2f} GB")
            print(f"  Allocated memory: {allocated_memory:.2f} GB")
            print(f"  Reserved memory: {reserved_memory:.2f} GB")
            print(f"  Available memory: {available_memory:.2f} GB")
    else:
        print("No GPU available.")

# we'll make use of this gradually to keep track of our GPU utilization
mem_status()

In [0]:
# first grab the DistilBERT model
from transformers import AutoModelForMaskedLM

model_checkpoint = "distilbert-base-uncased"
model = AutoModelForMaskedLM.from_pretrained(model_checkpoint)

In [0]:
# grab the tokenizer as well
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [0]:
# let's play around with DistilBERT's masked language model with an example before we fine-tune it on data related to movies
text = "This is a great [MASK]."

In [0]:
# given the text and its tokenization, which are the top 5 results likely to fill the [MASK] token as predicted by the pre-trained model?
import torch

inputs = tokenizer(text, return_tensors="pt")
token_logits = model(**inputs).logits
# find the location of [MASK] and extract its logits
mask_token_index = torch.where(inputs["input_ids"] == tokenizer.mask_token_id)[1]
mask_token_logits = token_logits[0, mask_token_index, :]
# pick the [MASK] candidates with the highest logits
top_5_tokens = torch.topk(mask_token_logits, 5, dim=1).indices[0].tolist()

for token in top_5_tokens:
    print(f"'>>> {text.replace(tokenizer.mask_token, tokenizer.decode([token]))}'")

# now lets fine-tune this model to give us more movie-specific responses

In [0]:
# load the large movie reviews dataset
from datasets import load_dataset

imdb_dataset = load_dataset("imdb")
imdb_dataset

In [0]:
# grab a random sample of the data
sample = imdb_dataset["train"].shuffle(seed=92).select(range(3))

for row in sample:
    print(f"\n'>>> Review: {row['text']}'")
    print(f"'>>> Label: {row['label']}'")

In [0]:
# for later pre-training tasks, we'll peek at the unsupervised split evident here in the dataset
sample = imdb_dataset["unsupervised"].shuffle(seed=92).select(range(3))

for row in sample:
    print(f"\n'>>> Review: {row['text']}'")
    print(f"'>>> Label: {row['label']}'")

In [0]:
# first, tokenizer our whole dataset without truncating before we concatenate the whole thing and split it evenly
def tokenize_function(examples):
    result = tokenizer(examples["text"])
    if tokenizer.is_fast:
        result["word_ids"] = [result.word_ids(i) for i in range(len(result["input_ids"]))]
    return result


# batched=True for multithreading, disperse of unnecessary label and text columns
tokenized_datasets = imdb_dataset.map(
    tokenize_function, batched=True, remove_columns=["text", "label"]
)
tokenized_datasets

In [0]:
# before chunking our text, we want to check out the max token context window for our model
tokenizer.model_max_length

In [0]:
# lets given 256 a try 🤷‍♂️
chunk_size = 256

In [0]:
# grab a small sample to design our chunk and concatenation method

# slicing produces a list of lists for each feature
tokenized_samples = tokenized_datasets["train"][:3]

for idx, sample in enumerate(tokenized_samples["input_ids"]):
    print(f"'>>> Review {idx} length: {len(sample)}'")

In [0]:
# use dict comprehension to create a dict of our concatenated samples
concatenated_examples = {
    k: sum(tokenized_samples[k], []) for k in tokenized_samples.keys()
}
total_length = len(concatenated_examples["input_ids"])
print(f"'>>> Concatenated reviews length: {total_length}'")

In [0]:
# now chunk this concatenation based on our maximum chunk size into nested lists of maximum chunk size length
chunks = {
    k: [t[i : i + chunk_size] for i in range(0, total_length, chunk_size)]
    for k, t in concatenated_examples.items()
}

for chunk in chunks["input_ids"]:
    print(f"'>>> Chunk length: {len(chunk)}'")

In [0]:
# with the last column of smaller length above, we can either drop it, or we can pad it to match the length of the others
# we'll take the padding approach as follows

# we'll pad to the input_ids col the tokenizer's pad_token_id, which we confirm with:
print(tokenizer.convert_ids_to_tokens(tokenizer.pad_token_id))

# similarly, each chunk is tokenized individually so the attention_mask for all tokens is 1, we'll just pad this col with 1s

# and lastly, in regards to word_ids col created by the tokenizer, the hugging face documentation states:
# 'A list indicating the word corresponding to each token. Special tokens added by the tokenizer are mapped to None and other tokens are mapped to the index of their corresponding word (several tokens will be mapped to the same word index if they are parts of that word).'
# so we will just tack on a bunch of Nones to match the max_chunk size

In [0]:
# we'll take the padding approach as follows, defining a function to be used in our preprocessing
def pad_last_chunk(examples, chunk_size):
  while len(examples["input_ids"][-1]) < chunk_size:
    examples["input_ids"][-1].append(tokenizer.pad_token_id)
  while len(examples["attention_mask"][-1]) < chunk_size:
    examples["attention_mask"][-1].append(1)
  while len(examples["word_ids"][-1]) < chunk_size:
    examples["word_ids"][-1].append(None)

pad_last_chunk(chunks, chunk_size)

# double check out lengths
for chunk in chunks["input_ids"]:
    print(f"'>>> input_ids chunk length: {len(chunk)}'")
for chunk in chunks["attention_mask"]:
    print(f"'>>> attention_mask chunk length: {len(chunk)}'")
for chunk in chunks["word_ids"]:
    print(f"'>>> word_ids chunk length: {len(chunk)}'")

In [0]:
# nice, now lets define our function for chunking, concatenating, and padding the whole dataset
def group_texts(examples):
    # concatenate all texts
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    # compute length of concatenated texts
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # Split by chunks of max_len
    result = {
        k: [t[i : i + chunk_size] for i in range(0, total_length, chunk_size)]
        for k, t in concatenated_examples.items()
    }
    # pad last column to match chunk_size
    pad_last_chunk(result, chunk_size)
    # create a new labels column
    result["labels"] = result["input_ids"].copy()
    return result

In [0]:
# now we apply this function by mapping it to our dataset
lm_datasets = tokenized_datasets.map(group_texts, batched=True)
lm_datasets

In [0]:
# its worth noting the following:
assert tokenized_datasets["train"].num_rows < lm_datasets["train"].num_rows
# which shows that upon chunking and concatenating the reviews, we now have examples involving 'contiguous tokens' that span across multiple examples from the original corpus. As such, our chunked and concatenated row count of labels is now larger than the row count of the original dataset of reviews. This is evident as indicated by the presence of special tokens like [CLS] [SEP] and [PAD]. 

# We can see this as follows:
print(tokenizer.decode(lm_datasets["train"][2313]["input_ids"]))

In [0]:
# before we train our model with a custom PyTorch training loop, we address the 'perplexity' metric used to evaluate our results
# https://en.wikipedia.org/wiki/Cross-entropy
# https://en.wikipedia.org/wiki/Perplexity

# mathematically, we define perplexity to be the exponential of our cross-entropy loss (or as the wiki states, 2**cross-loss entropy)
# intuitively, this tells us how "suprised" or "perplexed" the model is when seeing input tokens -  the lower the perplexity, the better our model is at predicting the next token

# we'll use a specific example from the gpt-2 model to demonstrate
import math
from transformers import AutoModelForCausalLM

gpt_model = AutoModelForCausalLM.from_pretrained("openai-community/gpt2")
gpt_tokenizer = AutoTokenizer.from_pretrained("openai-community/gpt2")

example_input = "The quick brown fox jumps over the lazy dog"
tokenized_example = gpt_tokenizer(example_input, return_tensors="pt")
# by passing the labels as the inputs themselves, the loss corresponds to the cross entropy between the input and output sequences
example_cross_entropy_loss = gpt_model(input_ids=tokenized_example["input_ids"], labels=tokenized_example["input_ids"]).loss

# compute the various forms of plexity for this example
print(f">>> Perplexity e^H(p,q): {math.exp(example_cross_entropy_loss):.2f}")
print(f">>> Perplexity 2^H(p,q): {2**(example_cross_entropy_loss):.2f}")

In [0]:
# verify we've used only the CPU so far before ultimately moving our model to the GPU for training
mem_status()

In [0]:
# now we define our data collator
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

In [0]:
# we can see how this datacollator works by comparing some examples
samples = [lm_datasets["train"][i] for i in range(2)]
for sample in samples:
    _ = sample.pop("word_ids")

for chunk in data_collator(samples)["input_ids"]:
    print(f"\n'>>> {tokenizer.decode(chunk)}'")

In [0]:
# while the above example shows indeed that our data collator is randomly masking tokens, we note that the default 'DataCollatorForLanguageModeling' does not allow for whole word masking - by default it masks just tokens, potentially masking in the middle of a word
# we construct a whole word masking function instead

import collections
import numpy as np

from transformers import default_data_collator

wwm_probability = 0.15


def whole_word_masking_data_collator(features):
    for feature in features:
        word_ids = feature.pop("word_ids")

        # Create a map between words and corresponding token indices
        mapping = collections.defaultdict(list)
        current_word_index = -1
        current_word = None
        for idx, word_id in enumerate(word_ids):
            if word_id is not None:
                if word_id != current_word:
                    current_word = word_id
                    current_word_index += 1
                mapping[current_word_index].append(idx)

        # Randomly mask words
        mask = np.random.binomial(1, wwm_probability, (len(mapping),))
        input_ids = feature["input_ids"]
        labels = feature["labels"]
        new_labels = [-100] * len(labels)
        for word_id in np.where(mask)[0]:
            word_id = word_id.item()
            for idx in mapping[word_id]:
                new_labels[idx] = labels[idx]
                input_ids[idx] = tokenizer.mask_token_id
        feature["labels"] = new_labels

    return default_data_collator(features)

In [0]:
# double check our function on the same samples as before
samples = [lm_datasets["train"][i] for i in range(2)]
batch = whole_word_masking_data_collator(samples)

for chunk in batch["input_ids"]:
    print(f"\n'>>> {tokenizer.decode(chunk)}'")

In [0]:
# before running our evaluation, we want to apply a single masking to the evaluation dataset
# if we let the masking be applied with each iteration of the training loop, there will be variation in how our evaluation dataset is masked each time, introducing some randomness in results perplexity scores
# to avoid this, we apply the masking a single time before training and iterative evaluation each epoch

# lastly, we will opt for the whole word masking approach and make use of our previous function

def insert_random_mask(batch):
    features = [dict(zip(batch, t)) for t in zip(*batch.values())]
    masked_inputs = whole_word_masking_data_collator(features)
    # Create a new "masked" column for each column in the dataset
    return {"masked_" + k: v.numpy() for k, v in masked_inputs.items()}

In [0]:
# we map this single masking onto the whole test dataset
eval_dataset = lm_datasets["test"].map(
    insert_random_mask,
    batched=True,
    remove_columns=lm_datasets["test"].column_names, # remove old columns now prefixed with masked_
)
# rename columns to match other splits
eval_dataset = eval_dataset.rename_columns(
    {
        "masked_input_ids": "input_ids",
        "masked_attention_mask": "attention_mask",
        "masked_labels": "labels",
    }
)
eval_dataset

In [0]:
# now we configure our dataloaders before pumping the data into the model for training and evaluation
from torch.utils.data import DataLoader
from transformers import default_data_collator

# previously OOMd with batch size 64
batch_size = 32

# for the train split we pass the custom whole word masking collator
train_dataloader = DataLoader(
    lm_datasets["train"],
    shuffle=True,
    batch_size=batch_size,
    collate_fn=whole_word_masking_data_collator,
)
# since we premasked our evaluation dataset to remove randomness, we simply use the default data collator
eval_dataloader = DataLoader(
    eval_dataset, batch_size=batch_size, collate_fn=default_data_collator
)

## Training

In [0]:
# we now load a fresh version of the model onto the GPU and check the mem status
model = AutoModelForMaskedLM.from_pretrained(model_checkpoint).to(torch.device("cuda"))
mem_status()

In [0]:
# now we define our optimizer as AdamW
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=5e-5)

In [0]:
# prepare everything with the 🤗 accelerate package
from accelerate import Accelerator

accelerator = Accelerator()
model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader
)

In [0]:
# configure a learning rate scheduler
from transformers import get_scheduler

num_train_epochs = 3
num_update_steps_per_epoch = len(train_dataloader)
num_training_steps = num_train_epochs * num_update_steps_per_epoch

lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)
num_training_steps

In [0]:
# check our memory status before we begin our loop -  we iteratively check it each epoch as training as well
mem_status()

In [0]:
# now we run our training loop!
from tqdm.auto import tqdm
import torch
import math

output_dir = "/Volumes/workspace_dogfood/jgr/hugging_face_cache/DistilBERT-DeNiro"
progress_bar = tqdm(range(num_training_steps))

for epoch in range(num_train_epochs):
    # training
    model.train()
    for batch in train_dataloader:
        # accelerate package handles this inherently, but I prefer to see it directly for future clarity
        batch = {k: v.to(accelerator.device) for k, v in batch.items()}
        # batch = {k: v.to(torch.device("cuda")) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        accelerator.backward(loss)
        # loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

    # evaluation
    model.eval()
    losses = []
    for step, batch in enumerate(eval_dataloader):
        # same note as above
        batch = {k: v.to(accelerator.device) for k, v in batch.items()}
        # batch = {k: v.to(torch.device("cuda")) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)

        loss = outputs.loss
        losses.append(accelerator.gather(loss.repeat(batch_size)))

    losses = torch.cat(losses)
    losses = losses[: len(eval_dataset)]
    try:
        perplexity = math.exp(torch.mean(losses))
    except OverflowError:
        perplexity = float("inf")

    print(f">>> Epoch {epoch}: Perplexity: {perplexity}")
    mem_status()

    # save and upload
    # first wait for all processes to reach the same stage
    accelerator.wait_for_everyone()
    # unwraps the model from accelerate.prepare() to reintroduce the save_pretrained() fn for saving
    unwrapped_model = accelerator.unwrap_model(model)
    # accelerator.save() instead of torch.save()
    unwrapped_model.save_pretrained(output_dir, save_function=accelerator.save)
    if accelerator.is_main_process:
        tokenizer.save_pretrained(output_dir)
        # unable to use this currently since we cannot clone this repo into our Databricks workspace
        # repo.push_to_hub(
        #     commit_message=f"Training in progress epoch {epoch}", blocking=False
        # )

## Post-training Evaluation 

In [0]:
mem_status()

In [0]:
# we ran out of memory when training the model with a batch size of 64
# we derive a rough estimate of the memory required for training a model as a function of its config

"""
A function for giving a rough estimate on the amount of memory required to train a model without activation checkpointing.

l: number of layers
p: precision
s: sequence length
b: batch size
h: hidden size
a: number of attention heads 
 """
def mem_required(l, p, s, b, h, a):
  total_bytes = l*p*s*b*h*(16+(2/p)+(2*a*s/h)+a*s/(p*h))
  return f"{total_bytes/(1024**3):.2f} GB"

In [0]:
# compute the mem required for training DistilBERT with our initial vs final choices
print("batch size 64 required ~ " + mem_required(6, 4, 256, 64, 3072, 12))
print("batch size 32 required ~ " + mem_required(6, 4, 256, 32, 3072, 12))

# this checks out - we only have a 16GB machine to train on

In [0]:
# comparing our domain-adapted DistilBERT-DeNiro to the standard DistilBERT after fine-tuning
# previously the standard DistilBERT model gave the following output on this text prompt in cell 6 above:
"""
'>>> This is a great deal.'
'>>> This is a great success.'
'>>> This is a great adventure.'
'>>> This is a great idea.'
'>>> This is a great feat.'

"""
# let's see how our newly trained model behaves!

text = "This is a great [MASK]."

inputs = tokenizer(text, return_tensors="pt")
token_logits = model(**inputs).logits

# find the location of [MASK] and extract its logits
mask_token_index = torch.where(inputs["input_ids"] == tokenizer.mask_token_id)[1]
mask_token_logits = token_logits[0, mask_token_index, :]

# pick the [MASK] candidates with the highest logits
top_5_tokens = torch.topk(mask_token_logits, 5, dim=1).indices[0].tolist()

for token in top_5_tokens:
    print(f"'>>> {text.replace(tokenizer.mask_token, tokenizer.decode([token]))}'")

In [0]:
# let's see how the fill mask works on a specific example with a little prompt engineering
text_2 = "[MASK] Tarantino is a really creative director!"

inputs_2 = tokenizer(text_2, return_tensors="pt")
token_logits_2 = model(**inputs_2).logits
mask_token_index_2 = torch.where(inputs_2["input_ids"] == tokenizer.mask_token_id)[1]
mask_token_logits_2 = token_logits_2[0, mask_token_index_2, :]

fill = torch.topk(mask_token_logits_2, 1, dim=1).indices[0].tolist()
print(text_2.replace(tokenizer.mask_token, tokenizer.decode([fill[0]])))

In [0]:
# looks like our model is still wrapped in a (HF? not PyTorch?) DDP by hugging face accelerate
model
# this prevents us from pushing it to the hub

In [0]:
# let's unwrap it
unwrapped_model = accelerator.unwrap_model(model)

In [0]:
# now sign into the hub and push our model
dbutils.widgets.text("hf_token", "", "hf_token")

In [0]:
hf_token = dbutils.widgets.get("hf_token")
!huggingface-cli login --token $hf_token

In [0]:
unwrapped_model.push_to_hub("DistilBERT-DeNiro")

In [0]:
# we could also have grabbed our model from the saved location for pushing like so:
trained_model = AutoModelForMaskedLM.from_pretrained("/Volumes/workspace_dogfood/jgr/hugging_face_cache/DistilBERT-DeNiro")

In [0]:
# detail how end users can use this model - for the model card
import torch
from transformers import AutoModelForMaskedLM, AutoTokenizer

model = AutoModelForMaskedLM.from_pretrained("MarioBarbeque/DistilBERT-DeNiro").to("cuda")
tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")

# Pass a unique string with a [MASK] token for the model to fill
text = "This is a great [MASK]!"

tokenized_text = tokenizer(text, return_tensors="pt").to("cuda")
token_logits = model(**tokenized_text).logits

mask_token_index = torch.where(tokenized_text["input_ids"] == tokenizer.mask_token_id)[1]
mask_token_logits = token_logits[0, mask_token_index, :]

top_5_tokens = torch.topk(mask_token_logits, 5, dim=1).indices[0].tolist()
for token in top_5_tokens:
  print(text.replace(tokenizer.mask_token, tokenizer.decode(token)))